# Conciliation products SGA - Infohub

## Contextualization:

### SGA send the file INI061 to load the details of customer and their corresponding products to Infohub. The file specification is described below:

#### DETAIL
<table>
<tr>
<td>TM-DATE</td><td>NUMBER</td><td>08</td>
</tr>
<tr>
<td>TM-HEURE</td><td>NUMBER</td><td>06</td>
</tr>
<tr>
<td>NO-CLI</td><td>NUMBER</td><td>08</td>
</tr>
<tr>
<td>NO-CPT-CLI</td><td>STRING</td><td>12</td>
</tr>

<tr>
<td>GR-BLOC-PONCT</td>
</tr>
<tr>
<td>NO-OFFRE-SPEC</td><td>NUMBER</td><td>05</td>
</tr>
 <tr>
<td>DT-DEB-OFFRE</td><td>NUMBER</td><td>08</td>
</tr>
<tr>
<td>DT-FIN-OFFRE</td><td>NUMBER</td><td>08</td>
</tr>
<tr>
<td>QTE-OFFRE-TOTAL</td><td>NUMBER</td><td>06</td>
</tr>
<tr>
<td>QTE-OFFRE-VAR</td><td>NUMBER</td><td>06</td>
</tr>
<tr>
<td>GR-PRODUCT</td>
</tr>
<tr>
<td>NO-PROD</td><td>NUMBER</td><td>05</td>
</tr>
 <tr>
<td>NO-SEQ-PROD</td><td>NUMBER</td><td>23</td>
</tr>
<tr>
<td>NO-SEQ-PARENT</td><td>NUMBER</td><td>16</td>
</tr>
<tr>
<td>TYP-PROD</td><td>NUMBER</td><td>03</td>
</tr>
<tr>
<td>CD-EXIS-COM</td><td>STRING</td><td>05</td>
</tr>
<tr>
<td>DT-DEB-PROD</td><td>STRING</td><td>08</td>
</tr>
<tr>
<td>DT-FIN-PROD</td><td>STRING</td><td>08</td>
</tr>
<tr>
<td>CODE-APPROV</td><td>STRING</td><td>30</td>
</tr>
<tr>
<td>NO-SERV-ASSOC</td><td>NUMBER</td><td>05</td>
</tr>
<tr>
<td>DESC-PROD-FR</td><td>STRING</td><td>30</td>
</tr>
<tr>
<td>DESC-PROD-EN</td><td>STRING</td><td>30</td>
</tr>
<tr>
<td>CARACT-PROD OCCURS 10</td>
</tr>
<tr>
<td>DESC-CARACT</td><td>STRING</td><td>30</td>
</tr>
 <tr>
<td>VAL-CARACT</td><td>STRING</td><td>64</td>
</tr>
<tr>
<td>NO-FICHIER-PERFTP</td><td>NUMBER</td><td>01</td>
</tr>
<tr>
<td>FILLER</td><td>STRING</td><td>21</td>
</tr>
</table>     

#### HEADER                      

<table>
<tbody>
<tr>
<td>HEADER-DATE</td><td>NUMBER</td><td>08</td>
</tr>
 <tr>
<td>HEADER-HEURE</td><td>NUMBER</td><td>06</td>
</tr>
<tr>
<td>FILLER</td><td>STRING</td><td>1186</td>
</tr>
</tbody>
</table>
                            
#### FOOTER     

<table>
<tbody>
<tr>
<td>FOOTER-NB-ENRG</td><td>NUMBER</td><td>08</td>
</tr>
 <tr>
<td>FOOTER-TYPE</td><td>NUMBER</td><td>05</td>
</tr>
<tr>
<td>FILLER</td><td>STRING</td><td>1187</td>
</tr>
</tbody>
</table>


### For dependency purposes, this notebook needs to be executed on the block order, any difference could result in a bad behaviour.

In [1]:
# Block 1
import pyodbc
import pandas as pd
from pandas.io import sql as psql
import numpy as np

## After importing the necessary packages, it imports the file to a dataframe (df_file). The next block also extracts a sorted list of customeraccountno to be used on the query

In [2]:
# Block 2
#Generates a convert to string function, will be used to convert each field in string
def conv_str(x):
    return str(x)

#Defines the file which will be read, it needs to be on the same directory as this notebook

input_file = 'conciliation_sga_012018.csv'

#Define columns size on the file
col_widths = [8, 6, 8, 12, 8, 5, 8, 8, 6, 6, 5, 23, 16, 3, 5, 8, 8, 30, 5, 30, 30, 30, 64]

#define column name to be named on dataframe
col_name_file = ['TM_DATE', 'TM_HEURE', 'NO_CLI', 'NO_CPT_CLI', 'ID_CPT_INET', 'NO_OFFRE_SPEC', 'DT_DEB_OFFRE', 'DT_FIN_OFFRE', 'QTE_OFFRE_TOTAL', 'QTE_OFFRE_VAR', 'NO_PROD', 'NO_SEQ_PROD', 'NO_SEQ_PARENT', 'TYP_PROD', 'CD_EXIS_COM', 'DT_DEB_PROD', 'DT_FIN_PROD', 'CODE_APPROV', 'NO_SERV_ASSOC_A', 'DESC_PROD_FR', 'DESC_PROD_EN', 'DESC_CAR', 'VAL_CAR']

#Convert all fields to string
col_conv = {'TM_DATE': conv_str, 'TM_HEURE': conv_str, 'NO_CLI': conv_str, 'NO_CPT_CLI': conv_str, 'ID_CPT_INET': conv_str, 'NO_OFFRE_SPEC': conv_str, 'DT_DEB': conv_str, 'DT_FIN': conv_str, 'QTE_OFFRE_TOTAL': conv_str, 'QTE_OFFRE_VAR': conv_str, 'NO_PROD': conv_str, 'NO_SEQ_PROD': conv_str, 'NO_SEQ_PARENT': conv_str, 'TYP_PROD': conv_str, 'CD_EXIS_COM': conv_str, 'DT_DEB_PROD': conv_str, 'DT_FIN_PROD': conv_str, 'CODE_APPROV': conv_str, 'NO_SERV_ASSOC_A': conv_str, 'DESC_PROD_FR': conv_str, 'DESC_PROD_EN': conv_str, 'DESC_CAR': conv_str, 'VAL_CAR': conv_str}

#Read file, filling the null as spaces
df_file = pd.read_fwf(input_file, skiprows=1, widths=col_widths, names=col_name_file,  converters=col_conv).fillna(" ")

#select only the columns that exists in our databases
col_name_db = ['NO_CLI', 'NO_CPT_CLI', 'ID_CPT_INET', 'NO_OFFRE_SPEC', 'DT_DEB_OFFRE', 'DT_FIN_OFFRE', 'QTE_OFFRE_TOTAL', 'QTE_OFFRE_VAR', 'NO_PROD', 'NO_SEQ_PROD', 'NO_SEQ_PARENT', 'TYP_PROD', 'CD_EXIS_COM', 'DT_DEB_PROD', 'DT_FIN_PROD', 'CODE_APPROV', 'NO_SERV_ASSOC_A', 'DESC_PROD_FR', 'DESC_PROD_EN']

df_file = df_file[col_name_db]

#sort unique values of customeraccountNumber and separate them by " and , 
fil_qry = df_file['NO_CPT_CLI'].drop_duplicates().sort_values().str.cat(sep='\',\'')
fil_qry = fil_qry[3:]+'\''

In [3]:
df_file.head()

,NO_CLI,NO_CPT_CLI,ID_CPT_INET,NO_OFFRE_SPEC,DT_DEB_OFFRE,DT_FIN_OFFRE,QTE_OFFRE_TOTAL,QTE_OFFRE_VAR,NO_PROD,NO_SEQ_PROD,NO_SEQ_PARENT,TYP_PROD,CD_EXIS_COM,DT_DEB_PROD,DT_FIN_PROD,CODE_APPROV,NO_SERV_ASSOC_A,DESC_PROD_FR,DESC_PROD_EN
0,53519876,535198760014,VLAAAALU,     ,        ,        ,      ,      ,714,VLAAAALU-1,,3,false,20170612,,05,1130,ACCES ILLICO.TV,ILLICO.TV ACCESS
1,53519876,535198760014,VLAAAALU,     ,        ,        ,      ,      ,738,VLAAAALU-2,,69,false,20170612,,05,0,CHAINES ILLICO,ILLICO CHANNELS
2,54765650,547656500014,VLAAABZO,     ,        ,        ,      ,      ,1017,VLAAABZO-2,,3,false,20171031,,01,3678,PLAN DE DEBRANCHEMENT,DISCONNECTION PLAN
3,54765650,547656500014,VLAAABZO,     ,        ,        ,      ,      ,1015,VLAAABZO-3,,90,false,20171031,,05,3827,DEBR SERVICE TV,DISC TV SERVICE
4,54765650,547656500014,VLAAABZO,     ,        ,        ,      ,      ,1016,VLAAABZO-4,,90,false,20171031,,05,3828,DEBR. SERVICE INTERNET,DISC. INTERNET SERVICE


In [ ]:
# Block 3
#connecting to your database through ODBC
cnxstr = 'DSN=IHP1;UID=paesdan;PWD=paeshp1'

### Query to extract records from IHP1 on the same layout as the database where processid 6 = SET CUSTOMERACCOUNTPRODUCT and 17 = SET_PRODUCTCAPACITY

In [ ]:
# Block 4
qry = """SELECT DISTINCT 
CUST.CUSTOMERID AS NO_CLI,
CUST.CUSTOMERACCOUNTNO AS NO_CPT_CLI,
FIRST_VALUE (CASE WHEN PDCT_CAR.CARACTERISTICNAME = \'ID_COMPTE_INTERNET\'    THEN PDCT_CAR.CARACTERISTICVALUE ELSE NULL END) IGNORE NULLS OVER (PARTITION BY PDCT.PRODUCTID, PDCT.PARENTPRODUCTID) AS ID_CPT_INET,
FIRST_VALUE (CASE WHEN PDCT_CAR.CARACTERISTICNAME = \'NUMERO_OFFRE_SPECIALE\' THEN PDCT_CAR.CARACTERISTICVALUE ELSE NULL END) IGNORE NULLS OVER (PARTITION BY PDCT.PRODUCTID, PDCT.PARENTPRODUCTID) AS NO_OFFRE_SPEC,
CASE WHEN FIRST_VALUE (CASE WHEN PDCT_CAR.CARACTERISTICNAME = \'NUMERO_OFFRE_SPECIALE\' THEN PDCT_CAR.CARACTERISTICVALUE ELSE NULL END) IGNORE NULLS OVER (PARTITION BY PDCT.PRODUCTID, PDCT.PARENTPRODUCTID) is not null THEN PDCT.PRODUCTEFFECTIVESTARTDATE ELSE NULL END AS DT_DEB_OFFRE,                                                                                                    
CASE WHEN FIRST_VALUE (CASE WHEN PDCT_CAR.CARACTERISTICNAME = \'NUMERO_OFFRE_SPECIALE\' THEN PDCT_CAR.CARACTERISTICVALUE ELSE NULL END) IGNORE NULLS OVER (PARTITION BY PDCT.PRODUCTID, PDCT.PARENTPRODUCTID) is not null THEN PDCT.PRODUCTEFFECTIVEENDDATE ELSE NULL END AS DT_FIN_OFFRE,                                                                                                    
FIRST_VALUE (CASE WHEN PDCT_CAR.CARACTERISTICNAME = \'QTE_OFFRE_TOTAL\'       THEN PDCT_CAR.CARACTERISTICVALUE ELSE NULL END) IGNORE NULLS OVER (PARTITION BY PDCT.PRODUCTID, PDCT.PARENTPRODUCTID) AS QTE_OFFRE_TOTAL,
FIRST_VALUE (CASE WHEN PDCT_CAR.CARACTERISTICNAME = \'QTE_OFFRE_VARIATION\'   THEN PDCT_CAR.CARACTERISTICVALUE ELSE NULL END) IGNORE NULLS OVER (PARTITION BY PDCT.PRODUCTID, PDCT.PARENTPRODUCTID) AS QTE_OFFRE_VAR,
FIRST_VALUE (CASE WHEN PDCT_CAR.CARACTERISTICNAME = \'SGA_NUMERO_COMPOSANTE\' THEN PDCT_CAR.CARACTERISTICVALUE ELSE NULL END) IGNORE NULLS OVER (PARTITION BY PDCT.PRODUCTID, PDCT.PARENTPRODUCTID) AS NO_PROD,
PDCT.PRODUCTID AS NO_SEQ_PROD,                                                                                     
PDCT.PARENTPRODUCTID AS NO_SEQ_PARENT,                                                                             
FIRST_VALUE (CASE WHEN PDCT_CAR.CARACTERISTICNAME = \'SGA_TYPE_COMPOSANTE\'   THEN PDCT_CAR.CARACTERISTICVALUE ELSE NULL END) IGNORE NULLS OVER (PARTITION BY PDCT.PRODUCTID, PDCT.PARENTPRODUCTID) AS TYP_PROD,
FIRST_VALUE (CASE WHEN PDCT_CAR.CARACTERISTICNAME = \'COMMANDE_EN_COURS\'     THEN PDCT_CAR.CARACTERISTICVALUE ELSE NULL END) IGNORE NULLS OVER (PARTITION BY PDCT.PRODUCTID, PDCT.PARENTPRODUCTID) AS CD_EXIS_COM, 
CASE WHEN FIRST_VALUE (CASE WHEN PDCT_CAR.CARACTERISTICNAME = \'NUMERO_OFFRE_SPECIALE\' THEN PDCT_CAR.CARACTERISTICVALUE ELSE NULL END) IGNORE NULLS OVER (PARTITION BY PDCT.PRODUCTID, PDCT.PARENTPRODUCTID) is null THEN PDCT.PRODUCTEFFECTIVESTARTDATE ELSE NULL END AS DT_DEB_PROD,                                                                                                    
CASE WHEN FIRST_VALUE (CASE WHEN PDCT_CAR.CARACTERISTICNAME = \'NUMERO_OFFRE_SPECIALE\' THEN PDCT_CAR.CARACTERISTICVALUE ELSE NULL END) IGNORE NULLS OVER (PARTITION BY PDCT.PRODUCTID, PDCT.PARENTPRODUCTID) is null THEN PDCT.PRODUCTEFFECTIVEENDDATE ELSE NULL END AS DT_FIN_PROD,
PDCT.PRODUCTSTATUS AS CODE_APPROV,                                                                                                     
FIRST_VALUE (CASE WHEN PDCT_CAR.CARACTERISTICNAME = \'SGA_NUMERO_SERVICE\'    THEN PDCT_CAR.CARACTERISTICVALUE ELSE NULL END) IGNORE NULLS OVER (PARTITION BY PDCT.PRODUCTID, PDCT.PARENTPRODUCTID) AS NO_SERV_ASSOC_A,
FIRST_VALUE (CASE WHEN PDCT_CAR.CARACTERISTICNAME = \'DESCRIPTION_FRANCAIS\'  THEN PDCT_CAR.CARACTERISTICVALUE ELSE NULL END) IGNORE NULLS OVER (PARTITION BY PDCT.PRODUCTID, PDCT.PARENTPRODUCTID) AS DESC_PROD_FR,
FIRST_VALUE (CASE WHEN PDCT_CAR.CARACTERISTICNAME = \'DESCRIPTION_ANGLAIS\'   THEN PDCT_CAR.CARACTERISTICVALUE ELSE NULL END) IGNORE NULLS OVER (PARTITION BY PDCT.PRODUCTID, PDCT.PARENTPRODUCTID) AS DESC_PROD_EN 
FROM IHPRODUCT PDCT, IHPRODUCTCARACTERISTIC PDCT_CAR, IHCUSTOMERACCOUNT CUST  WHERE 
PDCT_CAR.PRODUCTRECID = PDCT.PRODUCTRECID AND
CUST.CUSTOMERACCOUNTNO = PDCT.CUSTOMERACCOUNTNO AND 
CUST.IHEFFECTIVEENDTIMESTAMP IS NULL AND 
PDCT.IHEFFECTIVEENDTIMESTAMP IS NULL AND  
PDCT.CUSTOMERACCOUNTNO IN ("""+fil_qry+');'
qry = qry.replace('\n', ' ')

### Imports the data available to the max processlotid per customeraccountno. Since the max was not made by customer. It could be possible that not all the customeraccountno are on the query. The output is stored on a dataframe (df_db). 

In [ ]:
#Block 5
#Open a connection to the mentioned database
cnxn = pyodbc.connect(cnxstr)
#Open a cursor to input the data
df_db = psql.read_sql(qry, cnxn)
#close connection to database
cnxn.close()
df_db = df_db.fillna(0)

### Order both dataframes (DF_DB for infohub database data and DF_FILE for SGA data) by NO_CPT_CLI and ID_CPT_INET

In [ ]:
#block 6
df_db = df_db.sort_values(by=['NO_CPT_CLI', 'ID_CPT_INET'])
df_file = df_file.sort_values(by=['NO_CPT_CLI', 'ID_CPT_INET'])

In [ ]:
#block 7
df_db[['NO_CPT_CLI', 'ID_CPT_INET']].nunique() #Counts records on the database based on account and internet account  

In [ ]:
df_file[['NO_CPT_CLI', 'ID_CPT_INET']].nunique() # Count records on the file

### Since SGA can send data, which in our database is treated as historical data, for example, the offers received were already closed and purged on our database. For some cases it is Ok to have more rows on their file than in our database.

In [ ]:
df_file_match = df_file.where(df_file['NO_CPT_CLI'].isin(df_db['NO_CPT_CLI']))
df_file_match
#df_file_match.loc[3156]#['NO_CPT_CLI'].isin(df_db['NO_CPT_CLI'])
#df_file_match[~df_file_match['NO_CPT_CLI'].isin(df_db['NO_CPT_CLI'])].dropna()
#df_file_match[['NO_CPT_CLI', 'ID_CPT_INET']].nunique() 

In [ ]:
# to filter the records on the file:
#df_file[df_file['NO_CPT_CLI'] == '500023170011'] 
df_db['NO_OFFRE_SPEC'].drop_duplicates()

In [ ]:

#qry='SELECT * FROM IHPRODUCT WHERE OFFERID IS NOT NULL AND CUSTOMERACCOUNTNO IN ('+fil_qry+');' 
qry ="select * from ihproductcaracteristic where CUSTOMERACCOUNTNO = '500023170011' and CARACTERISTICNAME = 'NUMERO_OFFRE_SPECIALE';"
#Open a connection to the mentioned database
cnxn = pyodbc.connect(cnxstr)
#Open a cursor to input the data
cursor = cnxn.cursor()

#Generate dataframe based on the connection and query 
df_db = psql.read_sql(qry, cnxn)
#close connection to database
cnxn.close()
df_tst = df_db.fillna(0)

In [ ]:
df_tst